<a href="https://colab.research.google.com/github/ttogle918/NLU_3-/blob/main/%EC%B5%9C%EC%A7%80%ED%98%84_sts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **NLU - 문장 유사도 계산 (STS)**



- 과제 목표
  - 두 개의 한국어 문장을 입력받아 두 문장의 의미적 유사도를 출력
  - regression task ( 0 <= target <= 5 ) **=> klue 결과값이 0~5이다! logit을 정규화할 필요!**
    -  as a real value from 0 (no meaning overlap) to 5 (meaning equivalence)
    - [klue](https://klue-benchmark.com/tasks/67/overview/description)
- 학습 데이터 셋 ( 다운로드 가능 & 제공 예정 )
  - KLUE-STS
    - AIRBNB ( 리뷰 )
    - policy ( 뉴스 )
    - paraKOQC ( 스마트홈 쿼리 )
- 과제 결과물
  - 학습된 모델 ( 모델 자유 선택 ) ( train set만 사용해 학습 )
  - 학습 방식 보고서
    - 어떤 모델을 선택했나
    - 어떻게 파라미터를 튜닝했나
    - 어떤 훈련 과정을 거쳤는가
  - dev set score ( F1 )
  - 문장 유사도를 출력하는 API ( 프레임워크 자유 선택 )

- [graykode/ALBERT-Pytorch](https://github.com/graykode/ALBERT-Pytorch)
- [huggingface](https://huggingface.co/docs/transformers/model_doc/albert)


유사도 계산.. 순서는 상관없는 것 같다..

albert가 sop(문장 순서 예측)을 통해 모델을 훈련하기 때문에 사용하려고 했는데, 유사도만 계산하는 것이기 때문에 albert를 사용할 때 장점이 크지는 않을 것 같다.

해야할 부분 

1. 모델 바꿔서 해보기!

In [ ]:
# model load
!pip install pytorch-transformers

In [ ]:
!pip install transformers
!pip install datasets

In [1]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import AdamW
from torch.nn.utils import clip_grad_norm_
from torch.utils.data import Dataset, DataLoader
import numpy as np
from tqdm import tqdm, tqdm_notebook
from sklearn.metrics import f1_score
from scipy import stats
import time


In [2]:
# gpu 연산이 가능하면 'cuda:0', 아니면 'cpu' 출력
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device, torch.cuda.device_count()

(device(type='cuda', index=0), 1)

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
from transformers import BertForNextSentencePrediction, BertTokenizerFast, BertConfig
from transformers import AutoModel, AutoTokenizer
from transformers.optimization import get_cosine_schedule_with_warmup
from transformers import AdamW
from transformers import get_linear_schedule_with_warmup, get_constant_schedule

- "klue/roberta-large"
- "klue/roberta-small"
- "klue/roberta-base"
- "klue/bert-base"
- [klue에 등록된 모델](https://huggingface.co/klue)
- [한국어언어모델](https://littlefoxdiary.tistory.com/81)

# KLUE 데이터셋

[klue-sts-벤치마크-구조-보기](https://velog.io/@soyoun9798/KLUE-STS-%EB%B2%A4%EC%B9%98%EB%A7%88%ED%81%AC-%EA%B5%AC%EC%A1%B0-%EB%B3%B4%EA%B8%B0)


In [5]:
from datasets import load_dataset
dataset = load_dataset('klue', 'sts')

Reusing dataset klue (/root/.cache/huggingface/datasets/klue/sts/1.0.0/e0fc3bc3de3eb03be2c92d72fd04a60ecc71903f821619cb28ca0e1e29e4233e)


  0%|          | 0/2 [00:00<?, ?it/s]

In [6]:
print(f"type(dataset) : {type(dataset)}")
print(f"key : {dataset.keys()}")
print(f"type dataset[train] : {type(dataset['train'])}")
print(f"dataset[train] : {dataset['train']} \n\n")
# labels : { 이진분류 : 1, 반올림 값 : 3.7, 실제 label 값 : 3.71422... }
dataset['train'][0]

type(dataset) : <class 'datasets.dataset_dict.DatasetDict'>
key : dict_keys(['train', 'validation'])
type dataset[train] : <class 'datasets.arrow_dataset.Dataset'>
dataset[train] : Dataset({
    features: ['guid', 'source', 'sentence1', 'sentence2', 'labels'],
    num_rows: 11668
}) 




{'guid': 'klue-sts-v1_train_00000',
 'labels': {'binary-label': 1, 'label': 3.7, 'real-label': 3.714285714285714},
 'sentence1': '숙소 위치는 찾기 쉽고 일반적인 한국의 반지하 숙소입니다.',
 'sentence2': '숙박시설의 위치는 쉽게 찾을 수 있고 한국의 대표적인 반지하 숙박시설입니다.',
 'source': 'airbnb-rtt'}

In [7]:
# 데이터 10개만 확인
i = 0
for d in dataset['train'] :
  if i == 10 : break
  print(d)
  i += 1

{'guid': 'klue-sts-v1_train_00000', 'source': 'airbnb-rtt', 'sentence1': '숙소 위치는 찾기 쉽고 일반적인 한국의 반지하 숙소입니다.', 'sentence2': '숙박시설의 위치는 쉽게 찾을 수 있고 한국의 대표적인 반지하 숙박시설입니다.', 'labels': {'label': 3.7, 'real-label': 3.714285714285714, 'binary-label': 1}}
{'guid': 'klue-sts-v1_train_00001', 'source': 'policy-sampled', 'sentence1': '위반행위 조사 등을 거부·방해·기피한 자는 500만원 이하 과태료 부과 대상이다.', 'sentence2': '시민들 스스로 자발적인 예방 노력을\xa0한 것은 아산 뿐만이 아니었다.', 'labels': {'label': 0.0, 'real-label': 0.0, 'binary-label': 0}}
{'guid': 'klue-sts-v1_train_00002', 'source': 'paraKQC-sampled', 'sentence1': '회사가 보낸 메일은 이 지메일이 아니라 다른 지메일 계정으로 전달해줘.', 'sentence2': '사람들이 주로 네이버 메일을 쓰는 이유를 알려줘', 'labels': {'label': 0.3, 'real-label': 0.3333333333333333, 'binary-label': 0}}
{'guid': 'klue-sts-v1_train_00003', 'source': 'policy-sampled', 'sentence1': '긴급 고용안정지원금은 지역고용대응 등 특별지원금, 지자체별 소상공인 지원사업, 취업성공패키지, 청년구직활동지원금, 긴급복지지원제도 지원금과는 중복 수급이 불가능하다.', 'sentence2': '고용보험이 1차 고용안전망이라면, 국민취업지원제도는 2차 고용안전망입니다.', 'labels': {'label': 0.6, 'real-la

## dataset -> DataFrame

In [28]:
import pandas as pd

sentence1, sentence2, labels = [], [], []

for data in dataset['train'] :
  sentence1.append(data['sentence1'])
  sentence2.append(data['sentence2'])
  labels.append(data['labels']['real-label'])
  # labels.append(data['labels'])
df = pd.DataFrame({'sentence1' : sentence1, 'sentence2' : sentence2, 'labels' : labels})

In [29]:
df.head(3)

,sentence1,sentence2,labels
0,숙소 위치는 찾기 쉽고 일반적인 한국의 반지하 숙소입니다.,숙박시설의 위치는 쉽게 찾을 수 있고 한국의 대표적인 반지하 숙박시설입니다.,3.714286
1,위반행위 조사 등을 거부·방해·기피한 자는 500만원 이하 과태료 부과 대상이다.,시민들 스스로 자발적인 예방 노력을 한 것은 아산 뿐만이 아니었다.,0.000000
2,회사가 보낸 메일은 이 지메일이 아니라 다른 지메일 계정으로 전달해줘.,사람들이 주로 네이버 메일을 쓰는 이유를 알려줘,0.333333


# 모델 load test

In [50]:
model = BertForNextSentencePrediction.from_pretrained("klue/roberta-large")
tokenizer = AutoTokenizer.from_pretrained("klue/roberta-large")

You are using a model of type roberta to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.
Some weights of the model checkpoint at klue/roberta-large were not used when initializing BertForNextSentencePrediction: ['roberta.encoder.layer.20.output.dense.bias', 'roberta.encoder.layer.3.attention.output.dense.weight', 'roberta.embeddings.position_embeddings.weight', 'roberta.encoder.layer.7.attention.self.key.weight', 'roberta.encoder.layer.4.attention.self.value.bias', 'roberta.encoder.layer.23.output.dense.bias', 'roberta.encoder.layer.17.output.LayerNorm.bias', 'roberta.encoder.layer.9.output.LayerNorm.weight', 'roberta.encoder.layer.14.attention.self.value.bias', 'roberta.encoder.layer.20.attention.self.query.bias', 'roberta.encoder.layer.0.attention.self.query.bias', 'roberta.encoder.layer.19.attention.output.dense.bias', 'roberta.encoder.layer.9.attention.self.value.weight', 'roberta.encoder.layer.1.attention.self.key.bias

In [78]:
# 0과 2의 의미
tensorized_input = tokenizer('숙소 위치는 찾기 쉽고 일반적인 한국의 반지하 숙소입니다.',
 '숙박시설의 위치는 쉽게 찾을 수 있고 한국의 대표적인 반지하 숙박시설입니다.'
    )
tensorized_input

{'input_ids': [0, 9206, 4318, 2259, 1642, 2015, 1311, 2088, 3935, 31221, 3629, 2079, 10817, 2205, 9206, 12190, 18, 2, 8134, 10171, 2079, 4318, 2259, 1311, 2318, 1642, 2069, 1295, 1513, 2088, 3629, 2079, 3661, 31221, 10817, 2205, 8134, 10171, 12190, 18, 2], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [81]:
tokenizer.decode(tokenizer.encode('숙소 위치는 찾기 쉽고 일반적인 한국의 반지하 숙소입니다.',
 '숙박시설의 위치는 쉽게 찾을 수 있고 한국의 대표적인 반지하 숙박시설입니다.'))

'[CLS] 숙소 위치는 찾기 쉽고 일반적인 한국의 반지하 숙소입니다. [SEP] 숙박시설의 위치는 쉽게 찾을 수 있고 한국의 대표적인 반지하 숙박시설입니다. [SEP]'

In [73]:
outputs = model(
            **tensorized_input
        )
outputs.logits    # 같을 확률 / 아닐 확률

tensor([[-0.1542, -0.1047]], grad_fn=<AddmmBackward0>)

In [76]:
nn.functional.softmax(outputs.logits)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  """Entry point for launching an IPython kernel.


tensor([[0.4876, 0.5124]], grad_fn=<SoftmaxBackward0>)

In [108]:
tensorized_input = tokenizer(
        '숙소 위치는 찾기 쉽고 일반적인 한국의 반지하 숙소입니다.',
        '숙소 위치는 찾기 쉽고 일반적인 한국의 반지하 숙소입니다.',
        add_special_tokens=True,
        padding="longest",  # 배치내 가장 긴 문장을 기준으로 부족한 문장은 [PAD] 토큰을 추가
        truncation=True, # max_length를 넘는 문장은 이 후 토큰을 제거함
        max_length=512,
        return_tensors='pt' # 토크나이즈된 결과 값을 텐서 형태로 반환
    )
outputs = model(
            **tensorized_input
        )
outputs

NextSentencePredictorOutput([('logits',
                              tensor([[-0.1558, -0.1170]], grad_fn=<AddmmBackward0>))])

In [104]:
tensorized_input = tokenizer(
'새로운 친구들을 만나고 싶을 때 추천합니다.', 
'새로운 친구들을 만나고 싶을 때 추천합니다.',
        add_special_tokens=True,
        padding="longest",  # 배치내 가장 긴 문장을 기준으로 부족한 문장은 [PAD] 토큰을 추가
        truncation=True, # max_length를 넘는 문장은 이 후 토큰을 제거함
        max_length=512,
        return_tensors='pt' # 토크나이즈된 결과 값을 텐서 형태로 반환
    )
outputs = model(
            **tensorized_input, labels=torch.tensor([0])
        )
outputs

NextSentencePredictorOutput([('loss',
                              tensor(0.7599, grad_fn=<NllLossBackward0>)),
                             ('logits',
                              tensor([[-0.2058, -0.0764]], grad_fn=<AddmmBackward0>))])

# Dataset Tokenizing -> dataLoader

In [6]:
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler

In [7]:
class CustomDataset(Dataset):
    """
    문서 리스트를 받아 skip-gram 방식의 (target_word, context_word) 데이터 셋을 생성
    """
    def __init__(self, dataset):
        self.sentence1, self.sentence2, self.labels = self.make_dataset(dataset)
    
    def make_dataset(self, dataset):
        """
        self.label : dataset의 label의 list
        self.input : sentence1, sentence2를 tokenizer한 값을 이어 붙임 
        rlabels : # real-label
        """
        sentence1, sentence2, rlabels = [], [], []

        for data in dataset :
          rlabels.append(data['labels']['real-label'])
          sentence1.append(data['sentence1'])
          sentence2.append(data['sentence2'])
        return sentence1, sentence2, rlabels
        
    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.sentence1[idx], self.sentence2[idx], self.labels[idx]

In [8]:
def custom_collate_fn(batch):
    """
    - batch: list of tuples (input1_data(string), input2_data(string), target_data(int))
    
    한 배치 내 문장들을 tokenizing 한 후 텐서로 변환함. 
    이때, dynamic padding (즉, 같은 배치 내 토큰의 개수가 동일할 수 있도록, 부족한 문장에 [PAD] 토큰을 추가하는 작업)을 적용
    
    한 배치 내 레이블(target)은 텐서화 함.
    
    (input, target) 튜플 형태를 반환.
    """
    input1_list, input2_list, target_list = [], [], []
    
    for _input1, _input2, _target in batch:
        input1_list.append(_input1)
        input2_list.append(_input2)
        target_list.append(_target)
    
    tensorized_input = tokenizer(
        input1_list, input2_list,
        add_special_tokens=True,
        padding="longest",  # 배치내 가장 긴 문장을 기준으로 부족한 문장은 [PAD] 토큰을 추가
        truncation=True, # max_length를 넘는 문장은 이 후 토큰을 제거함
        max_length=512,
        return_tensors='pt' # 토크나이즈된 결과 값을 텐서 형태로 반환
    )
    
    tensorized_label = torch.tensor(target_list)
    
    return tensorized_input, tensorized_label

In [9]:
def make_dataloader(dataset, tok_model) :
  global tokenizer
  tokenizer = AutoTokenizer.from_pretrained(tok_model)
  train_dataset = CustomDataset(dataset['train'])
  valid_dataset = CustomDataset(dataset['validation'])

  train_dataloader = DataLoader(
      train_dataset,
      batch_size =32,
      sampler = RandomSampler(train_dataset),
      collate_fn = custom_collate_fn
  )
  valid_dataloader = DataLoader(
      valid_dataset,
      batch_size =32,
      sampler = SequentialSampler(valid_dataset),
      collate_fn = custom_collate_fn
  )
  print(type(train_dataloader))
  return train_dataloader, valid_dataloader

# model class

In [10]:
# 모델 클래스
class CustomSTS(BertForNextSentencePrediction):
    def __init__(self, hidden_size: int, model_name):
        self.bert_config = BertConfig.from_pretrained(
            model_name
        )   
        super(CustomSTS, self).__init__(self.bert_config)
             
        self.model = BertForNextSentencePrediction.from_pretrained(model_name, config=self.bert_config)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, input_ids=None, attention_mask=None, token_type_ids=None, labels=None):
        """
        outputs(NextSentencePredictorOutput) : logtis, loss(next_sentence_label이 주어질 때 return)
                hidden_states(optional), attentions(optional) 을 가지고 있다.
        loss는 주어진 label이 0~5 사이의 값으로 scale 되어있기 때문에 직접 구해야한다!
        """
        # logits's shape : (batch_size, 2)
        logits = self.model(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
        ).logits
        probs = self.softmax(logits)
        probs = probs[:, 0] * 5    # 0~5 사이의 값으로 정답(T)일 확률 뽑아내기
        return probs    # 정답(T)일 확률, 정답일때 1 

# train

### model, optimizer, scheduler 초기화

In [11]:
def initializer(train_dataloader, epochs=2, model_name='snunlp/KR-Medium'):
    """
    모델, 옵티마이저, 스케쥴러 초기화
    """
    model = CustomSTS(hidden_size=768, model_name=model_name)   # hidden size?

    optimizer = AdamW(
        model.parameters(), # update 대상 파라미터를 입력
        lr=2e-5,
        eps=1e-8
    )
    
    total_steps = len(train_dataloader) * epochs
    print(f"Total train steps with {epochs} epochs: {total_steps}")

    scheduler = get_linear_schedule_with_warmup(
        optimizer, 
        num_warmup_steps = 0, # 여기서는 warmup을 사용하지 않는다.
        num_training_steps = total_steps
    )
    return model, optimizer, scheduler

### checkpoint

In [12]:
def save_checkpoint(path, model, optimizer, scheduler, epoch, loss, f1):
    file_name = f'{path}/model{epoch}_loss:{loss:.4f}_f1:{f1:.4f}.ckpt'
    
    torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'scheduler_state_dict': scheduler.state_dict(),
            'loss' : loss,
            'f1' : f1
        }, 
        file_name
    )
    
    print(f"Saving epoch {epoch} checkpoint at {file_name}")

### train code

In [13]:
def train(model, train_dataloader, valid_dataloader=None, epochs=1):
        loss_fct = nn.MSELoss()

        before_loss, before_f1 = 1, 0
        
        for epoch in range(epochs) :

            print(f"*****Epoch {epoch} Train Start*****")
            # 배치 단위 평균 loss와 총 평균 loss 계산하기위해 변수 생성
            total_loss, batch_loss, batch_count = 0,0,0
            
            # model을 train 모드로 설정 & device 할당
            model.train()
            model.to(device)
            
            # data iterator를 돌면서 하나씩 학습
            for step, batch in enumerate(train_dataloader):
                batch_count+=1
                
                # tensor 연산 전, 각 tensor에 device 할당
                batch = tuple(item.to(device) for item in batch)
                
                batch_input, batch_label = batch
                
                # batch마다 모델이 갖고 있는 기존 gradient를 초기화/??
                model.zero_grad()
                
                # forward
                probs = model(**batch_input)

                # loss
                loss = loss_fct(probs, batch_label)
                batch_loss += loss.item()
                total_loss += loss.item()

                # pearsonr 상관계수
                # pearson = torch.corrcoef(torch.stack([probs, batch_label], dim=0))   #stats.pearsonr(probs, batch_label)[0]
                
                # f1-score
                f1 = f1_score([0 if p < 3 else 1 for p in batch_label], [0 if p < 3 else 1 for p in probs])

                # backward -> 파라미터의 미분(gradient)를 자동으로 계산
                loss.backward()

                # gradient clipping 적용 
                clip_grad_norm_(model.parameters(), 1.0)
                
                # optimizer & scheduler 업데이트
                optimizer.step()
                scheduler.step()
        
                # 그래디언트 초기화
                model.zero_grad()
        
                # 배치 64개씩 처리할 때마다 평균 loss와 lr를 출력
                if (step % 64 == 0 and step != 0):
                    learning_rate = optimizer.param_groups[0]['lr']
                    print(f"Epoch: {epoch}, Step : {step}, LR : {learning_rate:.10f}, Avg Loss : {batch_loss / batch_count:.4f}, f1 score : {f1:.4f}")

                    # 변수 초기화 
                    batch_loss, batch_count = 0,0


            print(f"Epoch {epoch} Total Mean Loss : {total_loss/(step+1):.4f}")
            print(f"*****Epoch {epoch} Train Finish*****\n")
            
            if valid_dataloader is not None:
                print(f"*****Epoch {epoch} Valid Start*****")
                valid_loss, valid_acc, valid_f1, valid_pearson = validate(model, valid_dataloader)
                print(f"Epoch {epoch} Valid Loss : {valid_loss:.4f} Valid Acc : {valid_acc:.2f} Valid f1 : {valid_f1:.4f}")
                print(f"pearson 상관 계수 ; {valid_pearson}")
                print(f"*****Epoch {epoch} Valid Finish*****\n")

            if before_loss > valid_loss :
                before_loss = valid_loss
                save_checkpoint("/content/drive/MyDrive/Colab Notebooks/nlp", model, optimizer, scheduler, epoch, valid_loss, valid_f1)

        print("Train Finished")

### validation code

In [14]:
def validate(model, valid_dataloader):
    loss_fct = nn.MSELoss()
    # 모델을 evaluate 모드로 설정 & device 할당
    model.eval()
    model.to(device)
    
    total_loss, total_acc, total_f1, total_pearson= 0,0, 0, 0
        
    for step, batch in enumerate(valid_dataloader):
        
        # tensor 연산 전, 각 tensor에 device 할당
        batch = tuple(item.to(device) for item in batch)
            
        batch_input, batch_label = batch
            
        # gradient 계산하지 않음
        with torch.no_grad():
            probs = model(**batch_input)
            
        # loss
        loss = loss_fct(probs, batch_label)
        total_loss += loss.item()
        
        # accuracy
        acc = 0
        for p, b in zip(probs, batch_label) :
          if (p > 3 and b > 3) or (p < 3 and b < 3 ) :
            acc += 1
        
        acc = acc / len(probs)
        total_acc+=acc
        
        # pearsonr 상관계수
        pearson = torch.corrcoef(torch.stack([probs, batch_label], dim=0))
        total_pearson += pearson

        # f1-score
        f1 = f1_score([0 if p < 3 else 1 for p in batch_label], [0 if p < 3 else 1 for p in probs])
        total_f1 += f1

    total_loss = total_loss/(step+1)
    total_acc = total_acc/(step+1)*100
    total_f1 = total_f1/(step+1)
    total_pearson = total_pearson/(step+1)
    return total_loss, total_acc, total_f1, total_pearson

### Train : 'snunlp/KR-Medium'

In [ ]:
train_dataloader, valid_dataloader = make_dataloader(dataset, "snunlp/KR-Medium")

In [25]:
epochs=20
model, optimizer, scheduler = initializer(train_dataloader, epochs, "snunlp/KR-Medium")

t0 = time.time()
train(model, train_dataloader, valid_dataloader, epochs)
print(time.time()-t0)

Some weights of the model checkpoint at snunlp/KR-Medium were not used when initializing BertForNextSentencePrediction: ['cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForNextSentencePrediction from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForNextSentencePrediction from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is depr

Total train steps with 20 epochs: 7300
*****Epoch 0 Train Start*****
Epoch: 0, Step : 64, LR : 0.0000, Avg Loss : 1.6681, f1 score : 0.9286
Epoch: 0, Step : 128, LR : 0.0000, Avg Loss : 0.3266, f1 score : 1.0000
Epoch: 0, Step : 192, LR : 0.0000, Avg Loss : 0.2903, f1 score : 0.8000
Epoch: 0, Step : 256, LR : 0.0000, Avg Loss : 0.2625, f1 score : 0.9167
Epoch: 0, Step : 320, LR : 0.0000, Avg Loss : 0.2438, f1 score : 1.0000
Epoch 0 Total Mean Loss : 0.5240
*****Epoch 0 Train Finish*****

*****Epoch 0 Valid Start*****
Epoch 0 Valid Loss : 0.8607 Valid Acc : 70.54 Valid f1 : 0.7144
*****Epoch 0 Valid Finish*****

Saving epoch 0 checkpoint at /content/drive/MyDrive/Colab Notebooks/nlp/model0_loss:0.5240_f1:0.7144.ckpt
*****Epoch 1 Train Start*****
Epoch: 1, Step : 64, LR : 0.0000, Avg Loss : 0.1896, f1 score : 1.0000
Epoch: 1, Step : 128, LR : 0.0000, Avg Loss : 0.1573, f1 score : 0.9333
Epoch: 1, Step : 192, LR : 0.0000, Avg Loss : 0.1561, f1 score : 0.9677
Epoch: 1, Step : 256, LR : 0.0

In [26]:
train(model, train_dataloader, valid_dataloader, 1)

*****Epoch 0 Train Start*****
Epoch: 0, Step : 64, LR : 0.0000, Avg Loss : 0.0108, f1 score : 1.0000
Epoch: 0, Step : 128, LR : 0.0000, Avg Loss : 0.0109, f1 score : 1.0000
Epoch: 0, Step : 192, LR : 0.0000, Avg Loss : 0.0105, f1 score : 0.9730
Epoch: 0, Step : 256, LR : 0.0000, Avg Loss : 0.0109, f1 score : 0.9730
Epoch: 0, Step : 320, LR : 0.0000, Avg Loss : 0.0109, f1 score : 1.0000
Epoch 0 Total Mean Loss : 0.0108
*****Epoch 0 Train Finish*****

*****Epoch 0 Valid Start*****
Epoch 0 Valid Loss : 0.5634 Valid Acc : 79.02 Valid f1 : 0.7773
*****Epoch 0 Valid Finish*****

Saving epoch 0 checkpoint at /content/drive/MyDrive/Colab Notebooks/nlp/model0_loss:0.0108_f1:0.7773.ckpt
Train Finished


In [47]:
if valid_dataloader is not None:
    valid_loss, valid_acc, valid_f1, valid_pearson = validate(model, valid_dataloader)
    print(f"Epoch 1 Valid Loss : {valid_loss:.4f} Valid Acc : {valid_acc:.2f} Valid f1 : {valid_f1:.4f}")
    print(f"pearson 상관 계수 ; {valid_pearson}")
    print(f"*****Epoch 1 Valid Finish*****\n")

Epoch 1 Valid Loss : 0.5634 Valid Acc : 79.02 Valid f1 : 0.7773
pearson 상관 계수 ; tensor([[1.0000, 0.8687],
        [0.8687, 1.0000]], device='cuda:0')
*****Epoch 1 Valid Finish*****



### Train : 'monologg/kobert' => X
[git](https://github.com/monologg/KoBERT-Transformers)

- sub task에 sts가 없음
- NSMC, NER, KorQuAD


In [17]:
train_dataloader, valid_dataloader = make_dataloader(dataset, 'monologg/kobert')
epochs=20
model, optimizer, scheduler = initializer(train_dataloader, epochs, 'monologg/kobert')   # 여기에 config 선언되어있다.

start = time.time()
train(model, train_dataloader, valid_dataloader, epochs)
end = time.time()
print(f"time : {(end - start)//60}분 {(end - start)%60}초")

Downloading:   0%|          | 0.00/51.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/426 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/76.0k [00:00<?, ?B/s]

<class 'torch.utils.data.dataloader.DataLoader'>


Downloading:   0%|          | 0.00/352M [00:00<?, ?B/s]

Some weights of BertForNextSentencePrediction were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Total train steps with 20 epochs: 7300
*****Epoch 0 Train Start*****
Epoch: 0, Step : 64, LR : 0.0000198219, Avg Loss : 2.6811, f1 score : 0.3158
Epoch: 0, Step : 128, LR : 0.0000196466, Avg Loss : 2.3828, f1 score : 0.5517
Epoch: 0, Step : 192, LR : 0.0000194712, Avg Loss : 2.3224, f1 score : 0.2857
Epoch: 0, Step : 256, LR : 0.0000192959, Avg Loss : 2.1728, f1 score : 0.3636
Epoch: 0, Step : 320, LR : 0.0000191205, Avg Loss : 2.0097, f1 score : 0.6667
Epoch 0 Total Mean Loss : 2.2716
*****Epoch 0 Train Finish*****

*****Epoch 0 Valid Start*****
Epoch 0 Valid Loss : 2.7981 Valid Acc : 56.01 Valid f1 : 0.4805
pearson 상관 계수 ; tensor([[1.0000, 0.1509],
        [0.1509, 1.0000]], device='cuda:0')
*****Epoch 0 Valid Finish*****

*****Epoch 1 Train Start*****
Epoch: 1, Step : 64, LR : 0.0000188219, Avg Loss : 1.8499, f1 score : 0.6250
Epoch: 1, Step : 128, LR : 0.0000186466, Avg Loss : 1.8260, f1 score : 0.4615
Epoch: 1, Step : 192, LR : 0.0000184712, Avg Loss : 1.8183, f1 score : 0.8800
Ep

### train : 'klue/bert-base'

- Epoch 15 : 가장 작은 valid_loss를 가지고 있다.
  - Valid Loss : 0.3798
  - Valid Acc : 81.75
  - Valid f1 : 0.8095
  - pearson 상관 계수 : [[1.0000, 0.9116],[0.9116, 1.0000]]


In [18]:
train_dataloader, valid_dataloader = make_dataloader(dataset, 'klue/bert-base')
epochs=20
model, optimizer, scheduler = initializer(train_dataloader, epochs, 'klue/bert-base')   # 여기에 config 선언되어있다.

start = time.time()
train(model, train_dataloader, valid_dataloader, epochs)
end = time.time()
print(f"time : {(end - start)//60}분 {(end - start)%60}초")

Downloading:   0%|          | 0.00/289 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/425 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/243k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/125 [00:00<?, ?B/s]

<class 'torch.utils.data.dataloader.DataLoader'>


Downloading:   0%|          | 0.00/424M [00:00<?, ?B/s]

Some weights of the model checkpoint at klue/bert-base were not used when initializing BertForNextSentencePrediction: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForNextSentencePrediction from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForNextSentencePrediction from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprec

Total train steps with 20 epochs: 7300
*****Epoch 0 Train Start*****
Epoch: 0, Step : 64, LR : 0.0000198219, Avg Loss : 1.3634, f1 score : 0.7857
Epoch: 0, Step : 128, LR : 0.0000196466, Avg Loss : 0.2565, f1 score : 0.9714
Epoch: 0, Step : 192, LR : 0.0000194712, Avg Loss : 0.2213, f1 score : 0.9677
Epoch: 0, Step : 256, LR : 0.0000192959, Avg Loss : 0.2065, f1 score : 0.9565
Epoch: 0, Step : 320, LR : 0.0000191205, Avg Loss : 0.1954, f1 score : 0.8966
Epoch 0 Total Mean Loss : 0.4213
*****Epoch 0 Train Finish*****

*****Epoch 0 Valid Start*****
Epoch 0 Valid Loss : 0.6604 Valid Acc : 76.97 Valid f1 : 0.7891
pearson 상관 계수 ; tensor([[1.0000, 0.8633],
        [0.8633, 1.0000]], device='cuda:0')
*****Epoch 0 Valid Finish*****

Saving epoch 0 checkpoint at /content/drive/MyDrive/Colab Notebooks/nlp/model0_loss:0.6604_f1:0.7891.ckpt
*****Epoch 1 Train Start*****
Epoch: 1, Step : 64, LR : 0.0000188219, Avg Loss : 0.1142, f1 score : 0.9600
Epoch: 1, Step : 128, LR : 0.0000186466, Avg Loss : 